<a href="https://colab.research.google.com/github/vsingh9076/Generative_AI/blob/main/Semantic_Search/Vector_Database_Pinecone_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Credit : [YouTube](https://www.youtube.com/watch?v=bWOvO_cxLHw)

**Vector databases** are purpose-built to handle the unique structure of **vector embeddings**. They **index vectors** for **easy search and retrieval** by comparing values and finding those that are most similar to one another.

In [1]:
!pip install sentence-transformers pinecone-client -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 583.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 16.7 MB/s eta 0:00:00


In [2]:
import pandas as pd

In [3]:
# https://www.kaggle.com/datasets/quora/question-pairs-dataset
data = pd.read_csv("/content/questions.csv")
data.head()

ParserError: ignored

In [ ]:
len(data)

199

In [ ]:
data[data['is_duplicate']==1].head()

,id,qid1,qid2,question1,question2,is_duplicate
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1


In [ ]:
import pinecone
pinecone.init(api_key='1ef90430-72dd-46d7-919a-63620484b009', environment='us-west1-gcp')

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2',device='cuda')

In [ ]:
embeding = model.encode("This is sentence")
len(embeding)

768

In [ ]:
pinecone.create_index(name='question-search', dimension=768)

In [ ]:
index = pinecone.Index('question-search')

In [ ]:
# [(id,vector,metadata)]

In [ ]:
question_list = []
for i,row in data.iterrows():
  question_list.append(
      (
        str(row['id']),
        model.encode(row['question1']).tolist(),
        {
            'is_duplicate': int(row['is_duplicate']),
            'question1': row['question1']
        }
      )
  )
  if len(question_list)==50 or len(question_list)==len(data):
    index.upsert(vectors=question_list)
    question_list = []

In [ ]:
query = "How do I prepare for civil service?"
xq = model.encode([query]).tolist()
result = index.query(xq, top_k=2, includeMetadata=True)
result

{'matches': [{'id': '38',
              'metadata': {'is_duplicate': 1.0,
                           'question1': 'How do we prepare for UPSC?'},
              'score': 0.617168427,
              'values': []},
             {'id': '29',
              'metadata': {'is_duplicate': 1.0,
                           'question1': 'How should I prepare for CA final '
                                        'law?'},
              'score': 0.611709058,
              'values': []}],
 'namespace': ''}

In [ ]:
query = "How to levergae internet for business"
xq = model.encode([query]).tolist()
result = index.query(xq, top_k=2, includeMetadata=True)
result

{'matches': [{'id': '78',
              'metadata': {'is_duplicate': 0.0,
                           'question1': 'How can I make money through the '
                                        'Internet?'},
              'score': 0.473660648,
              'values': []},
             {'id': '28',
              'metadata': {'is_duplicate': 0.0,
                           'question1': 'What is best way to make money '
                                        'online?'},
              'score': 0.44515273,
              'values': []}],
 'namespace': ''}